# Database engineering Jupyter Notebook to import, clean and export Surfs-Up! data

In [1]:
# Dependency setup
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey

In [2]:
# Defining the path for the pandas CSV imports
clean_meas_csv = "data_files/CLEAN_hawaii_measurements.csv"
clean_stat_csv = "data_files/CLEAN_hawaii_stations.csv"

In [3]:
# Reading csv data into pandas dataframe
measurements_df = pd.read_csv(clean_meas_csv, header="infer",dtype={"station":"str","date":"str","prcp":"float","tobs":"float"})
measurements_df["date"] = pd.to_datetime(measurements_df["date"],format="%Y-%m-%d", errors="coerce")
stations_df = pd.read_csv(clean_stat_csv, header="infer", dtype={"station":"str","name":"str","latitude":"float","longitude":"float","elevation":"float"},index_col="station")

In [4]:
# Creating class for the "stations" table
class Stations(Base):
    __tablename__="stations"
    station = Column(String(50), primary_key = True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
# Creating dtype dictionary for Stations data
dict_stations = {"station":sa.String(50),
                 "name": sa.String(255),
                 "latitude":sa.Float(),
                 "longitude":sa.Float(),
                 "elevation":sa.Float(),
                }

In [5]:
# Creating class for the "measurements" table
class Measurements(Base):
    __tablename__="measurements"
    id = Column(Integer, primary_key = True)
    station = Column(String(50), ForeignKey("stations.station"))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
# Creating dtype dictionary Measurements Data
dict_measurements = {"id":sa.Integer(),
                     "station":sa.String(50),
                     "date":sa.Date(),
                     "prcp":sa.Float(),
                     "tobs":sa.Float(),
                    }


In [6]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///hawaii.sqlite')

In [7]:
# Creating the SQLITE data tables from the previously created engine
Base.metadata.create_all(engine)

In [8]:
# Establishing pathway for dataframe to be inserted into SQLITE database
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [9]:
# Exporting pandas measurement dataframe to hawaii.sqlite database
measurements_df.to_sql(name="measurements", con=engine, if_exists="replace",index=True, dtype=dict_measurements)

In [10]:
# Exporting pandas measurement dataframe to hawaii.sqlite database
stations_df.to_sql(name="stations", con=engine, if_exists="replace",index=True, dtype=dict_stations)

In [11]:
# Writing data to file
session.commit()

In [12]:
# Closing session
session.close()

In [13]:
# Query to confirm measurement data was added to hawaii.sqlite
engine.execute("SELECT * FROM measurements LIMIT 1").fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65.0)]

In [14]:
# Query to confirm station data was added to hawaii.sqlite
engine.execute("SELECT * FROM stations LIMIT 1").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)]